# Features

## 1. long-term position control
## 2. B/S number per trade
## 3. KDJ risk control
## 4. trade log

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from main import Choice

In [2]:
zz1000 = Choice(2.073, 3.247, 20000, "512100.SH")
kc50 = Choice(0.848, 1.696, 50000, "588000.SH")
gqhl = Choice(1.820, 2.215, 30000, "501059.SH")
jyyh = Choice(3.67, 5.38, 13000, "002807.SZ")
wxyh = Choice(4.80, 7.06, 10000, "600908.SH")
yyetf = Choice(0.454, 0.907, 100000, "512010.SH")

512100.SH: buy_price_position
	price: 3.247, position: 0.000
	price: 2.777, position: 0.100
	price: 2.496, position: 0.300
	price: 2.256, position: 0.600
	price: 2.052, position: 1.000
512100.SH: sell_price_position
	price: 3.334, position: 0.000
	price: 2.928, position: 0.400
	price: 2.590, position: 0.700
	price: 2.308, position: 0.900
	price: 2.073, position: 1.000
588000.SH: buy_price_position
	price: 1.696, position: 0.000
	price: 1.357, position: 0.100
	price: 1.153, position: 0.300
	price: 0.980, position: 0.600
	price: 0.833, position: 1.000
588000.SH: sell_price_position
	price: 1.759, position: 0.000
	price: 1.465, position: 0.400
	price: 1.221, position: 0.700
	price: 1.018, position: 0.900
	price: 0.848, position: 1.000
501059.SH: buy_price_position
	price: 2.215, position: 0.000
	price: 2.057, position: 0.100
	price: 1.962, position: 0.300
	price: 1.882, position: 0.600
	price: 1.813, position: 1.000
501059.SH: sell_price_position
	price: 2.244, position: 0.000
	price: 2.1

## 1. long-term position control

In [6]:
choices = [zz1000, kc50, gqhl, jyyh, wxyh, yyetf]
max_pos = [20000, 50000, 30000, 13000, 10000, 100000]
prices  = [2.595, 1.032, 1.928, 4.050, 5.440, 0.491]
for choice, price, m_pos in zip(choices, prices, max_pos):
    long_pos = choice.cal_position(price, True)
    short_pos = choice.cal_position(price, False)
    print(f"{choice.code}: price: {price:.3f},\n\t"
          f"long position: {long_pos:.3f}, {long_pos * m_pos:.0f}\n\t"
          f"short position: {short_pos:.3f}, {short_pos * m_pos:.0f}")


512100.SH: price: 2.595,
	long position: 0.229, 4589
	short position: 0.695, 13903
588000.SH: price: 1.032,
	long position: 0.510, 25516
	short position: 0.886, 44292
501059.SH: price: 1.928,
	long position: 0.427, 12820
	short position: 0.839, 25165
002807.SZ: price: 4.050,
	long position: 0.503, 6534
	short position: 0.881, 11459
600908.SH: price: 5.440,
	long position: 0.413, 4130
	short position: 0.831, 8307
512010.SH: price: 0.491,
	long position: 0.771, 77086
	short position: 0.959, 95916


## 2. buy/sell number per trade

#### 1. standard situation
set 1/40=2.5%

if trade while 3% decrease with 1/40 = 2.5% position, 
from top 100% price to bottom 50% price, we should trade about 23 times(0.97^23=0.51), we get 23*2.5% = 57.5% position

that seems ok because KDJ will hint us and the buy diff will decrease from 3% to 1%.

if trade every 1%, trade will happen about 68 times(0.99^68=0.50)

#### 2. scaled situation
set 1/40*scale=2.5%*scale, scale=ln(0.5)/ln(bottom_price)

from top 100% price to bottom 68% price, 3% diff will trade about 13 times($0.97^{13}=0.67$), we get $13*2.5% = 32.5%$ position; 1% diff will trade about 38 times($0.99^38=0.68$), we get $38*2.5%=95.0%$ position.

price range decreased $64%=(1-0.68)/(1-0.5)$
trade number decreased $43%=(23-13)/23$ and $44%=(68-38)/68$

To get similar position, we should trade more: 
trade number should increased: $77%=(23-13)/13$ and $79%=(68-38)/38$

summerize the theoretical formula:
- $0.97^{times_{standard}} = 0.5$
- $0.97^times_scaled   = bottom_price$
- $scale = times_standard / times_scaled$

so, we get:
- $ln(0.97) * times_standard = ln(0.5)$
- $ln(0.97) * times_scaled   = ln(bottom_price)$
- $scale = ln(0.5) / ln(bottom_price)$

for bottom_price = 0.68, $scale = ln(0.5) / ln(0.68) = 1.80$, consistent with the above calculation 77% and 79%.

In [8]:
for choice in choices:
    num_per_trade = choice.cal_pos_number_per_trade()
    print(f"{choice.code}: number_per_trade: {num_per_trade:.0f}")

512100.SH: number_per_trade: 772
588000.SH: number_per_trade: 1250
501059.SH: number_per_trade: 2647
002807.SZ: number_per_trade: 589
600908.SH: number_per_trade: 449
512010.SH: number_per_trade: 2504


## 3. KDJ risk control

In [9]:
kdj_d_month = [52, 32, 53, 43, 41, 25]
kdj_d_week  = [49, 41, 51, 39, 44, 66]
for choice, d_month, d_week, price in zip(choices, kdj_d_month, kdj_d_week, prices):
    risk_density = choice.cal_risk_and_grid_density(d_month, d_week)
    print(f"{choice.code}: price: {price:.3f},\n\t"
          f"risk level: {risk_density[0]}\n\t"
          f"buy sell contrast and diff: {risk_density[1:]}")

512100.SH: price: 2.595,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)
588000.SH: price: 1.032,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)
501059.SH: price: 1.928,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)
002807.SZ: price: 4.050,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)
600908.SH: price: 5.440,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)
512010.SH: price: 0.491,
	risk level: 很大机会
	buy sell contrast and diff: (0.03, 0.02, 0.04, 0.03)


## 4. trade log

- add a trade transaction
- load from csv log
- save to csv log
